In [1]:
import os
import sys
from bs4 import BeautifulSoup
import pymongo
import datetime
import urllib2
import json

import sys
sys.path.insert(0, '../')
from magpie import MagpieModel
from keras.models import load_model
from magpie.utils import load_from_disk, save_to_disk
import gensim
import os
import numpy as np
import sklearn.metrics as metrics

Using TensorFlow backend.


In [2]:
client = pymongo.MongoClient('mongodb://um.media.mit.edu:27017/super-glue')
db = client.nyt_corpus
articles_collection = db.test_articles
docs = []
db_articles = articles_collection.find({})
for doc in db_articles:
    docs.append(doc)

In [3]:
repo_dir = "../word2vec-GoogleNews-vectors"
file_name = "GoogleNews-vectors-negative300.bin"
file_path = os.path.join(repo_dir, file_name)

In [4]:
print "Loading pre-trained word to vec model..."
word2vecmodel = gensim.models.Word2Vec.load_word2vec_format(file_path, binary=True)
print "weord2vec Model loaded"

Loading pre-trained word to vec model...
weord2vec Model loaded


In [5]:
labels3000 = []
with open('../models/labels_long.json') as data_file:
    labels3000 = json.load(data_file)
scaler3000 = load_from_disk('../scaler/scaler_labels_long')
keras_model3000 = load_model('../models/saved_models_labels_long/weights.01-0.00.hdf5')
model3000 = MagpieModel(keras_model=keras_model3000, word2vec_model=word2vecmodel, scaler=scaler3000, labels=labels3000)

labels600 = []
with open('../models/labels.json') as data_file:
    labels600 = json.load(data_file)
scaler600 = load_from_disk('../scaler/scaler')
keras_model600 = load_model('../models/saved_models/trained_model.h5')
model600 = MagpieModel(keras_model=keras_model600, word2vec_model=word2vecmodel, scaler=scaler600, labels=labels600)

labels_all = []
with open('../models/descriptors.json') as data_file:
    labels_all = [l["word"] for l in json.load(data_file)]
scaler_all = load_from_disk('../scaler/scalar_all_labels')
keras_model_all = load_model('../models/saved_models_all_labels/trained_model_all_labels.h5')
model_all = MagpieModel(keras_model=keras_model_all, word2vec_model=word2vecmodel, scaler=scaler_all, labels=labels_all)

labels_tax = []
with open('../models/labels_with_taxonomies.json') as data_file:
    labels_tax = json.load(data_file)
scaler_tax = load_from_disk('../scaler/scalar_all_labels')
keras_model_tax = load_model('../models/saved_models_with_taxonomies/weights.01-0.00.hdf5')
model_tax = MagpieModel(keras_model=keras_model_tax, word2vec_model=word2vecmodel, scaler=scaler_tax, labels=labels_tax)

In [8]:
true_1291 = np.zeros(len(labels3000))
for d in docs[1291]['descriptors']:
    if d.lower() in labels3000:
        true_1291[labels3000.index(d.lower())] = 1

In [15]:
from magpie.base.document import Document
def _predict(model, text):
    """
    Predict labels for a given Document object
    :param doc: Document object
    :return: list of labels with corresponding confidence intervals
    """
    doc = Document(0, None, text=text)
    if type(model.keras_model.input) == list:
        _, sample_length, embedding_size = model.keras_model.input_shape[0]
    else:
        _, sample_length, embedding_size = model.keras_model.input_shape

    words = doc.get_all_words()[:sample_length]
    x_matrix = np.zeros((1, sample_length, embedding_size))

    for i, w in enumerate(words):
        if w in model.word2vec_model:
            word_vector = model.word2vec_model[w].reshape(1, -1)
            scaled_vector = model.scaler.transform(word_vector, copy=True)[0]
            x_matrix[doc.doc_id][i] = scaled_vector

    if type(model.keras_model.input) == list:
        x = [x_matrix] * len(model.keras_model.input)
    else:
        x = [x_matrix]

    y_predicted = model.keras_model.predict(x)
    return y_predicted

In [31]:
models = [
    {"model": model3000, "labels":labels3000, "name":"model 3000"},
    {"model": model600, "labels":labels600, "name":"model 600"},
    {"model": model_all, "labels":labels_all, "name":"model all"},
    {"model": model_tax, "labels":labels_tax, "name":"model tax"},
         ]

In [70]:
for model in models[1:]:
    # Create true matrix
    # create pred matrix
    y_true = np.zeros((len(docs), len(model["labels"])))
    y_pred = np.zeros((len(docs), len(model["labels"])))
    print model["name"]
    print "-----"
    for i,doc in enumerate(docs):
        count = 0
        for descriptor in doc['descriptors']:
            if descriptor.lower() in model["labels"]:
                count+=1
                y_true[i][model["labels"].index(descriptor.lower())] = 1.0
        if doc["full_text"]:
            y_pred[i] = _predict(model["model"], doc["full_text"])
        if i%(len(docs)/10)==0:
            print "finished %d percent"%(i/(len(docs)/10)*10)

    y_pred_bin = np.where(y_pred>0.01, 1, 0)
    model["y_true"] = y_true
    model["y_pred"] = y_pred
    model["y_pred_bin"] = y_pred_bin
    model["coverage_error"] = metrics.coverage_error(y_true, y_pred)
    model ["label_ranking_average_precision_score"] = metrics.label_ranking_average_precision_score(y_true, y_pred)
    model ["label_ranking_loss"] = metrics.label_ranking_loss(y_true, y_pred)

model 600
-----
finished 0 percent
finished 10 percent
finished 20 percent
finished 30 percent
finished 40 percent
finished 50 percent
finished 60 percent
finished 70 percent
finished 80 percent
finished 90 percent
finished 100 percent
model all
-----
finished 0 percent
finished 10 percent
finished 20 percent
finished 30 percent
finished 40 percent
finished 50 percent
finished 60 percent
finished 70 percent
finished 80 percent
finished 90 percent
finished 100 percent
model tax
-----
finished 0 percent
finished 10 percent
finished 20 percent
finished 30 percent
finished 40 percent
finished 50 percent
finished 60 percent
finished 70 percent
finished 80 percent
finished 90 percent
finished 100 percent


In [72]:
for model in models:
    print model["name"]
    print "------------"
    print "coverage error: %f"%model["coverage_error"]
    print "average precision: %f"%model["label_ranking_average_precision_score"]
    print "label ranking loss: %f"%model["label_ranking_loss"]
    print

model 3000
------------
coverage error: 227.611252
average precision: 0.487446
label ranking loss: 0.049099

model 600
------------
coverage error: 46.469027
average precision: 0.590366
label ranking loss: 0.057090

model all
------------
coverage error: 1765.057838
average precision: 0.486731
label ranking loss: 0.049831

model tax
------------
coverage error: 428.873262
average precision: 0.320599
label ranking loss: 0.052623



In [140]:
# print np.isfinite(models[2]["y_true"]).all()
# print np.isfinite(models[2]["y_pred"]).all()
i=0
models[i]["y_pred_bin"] = np.where(models[i]["y_pred"]>0.005, 1, 0)
print metrics.classification_report(models[i]["y_true"], models[i]["y_pred_bin"])
# print metrics.precision_score(models[0]["y_true"], models[0]["y_pred_bin"], average='macro')
# print models[2]["label_ranking_average_precision_score"]
# print models[2]["label_ranking_loss"]

             precision    recall  f1-score   support

          0       0.05      1.00      0.09        93
          1       0.01      1.00      0.01        14
          2       0.08      0.98      0.15        86
          3       0.08      1.00      0.15        10
          4       0.13      0.96      0.22       171
          5       0.07      0.96      0.12        48
          6       0.15      0.84      0.25        87
          7       0.09      0.92      0.17        84
          8       0.19      0.98      0.32        41
          9       0.03      0.87      0.07        31
         10       0.10      0.98      0.18        66
         11       0.61      0.98      0.75        94
         12       0.01      0.89      0.02         9
         13       0.01      0.25      0.01         4
         14       0.00      1.00      0.00         2
         15       0.03      0.94      0.06        33
         16       0.02      1.00      0.04        27
         17       0.06      0.88      0.11   

In [25]:
true_1291_t

array([[ 0.,  0.,  1., ...,  0.,  0.,  0.]])

In [27]:
ce = metrics.coverage_error(true_1291_t, pred_1291)

In [29]:
label_ranking_average_precision_score = metrics.label_ranking_average_precision_score(true_1291_t, pred_1291)
label_ranking_loss = metrics.label_ranking_loss(true_1291_t, pred_1291)

In [30]:
print label_ranking_average_precision_score
print label_ranking_loss


0.555555555556
0.000482811896485


In [37]:
metrics.r2_score(true_1291_t, pred_1291)

/Users/jasrub/Documents/Git/nyt_classify_data/venv/lib/python2.7/site-packages/sklearn/metrics/regression.py:482: DeprecationWarning: Default 'multioutput' behavior now corresponds to 'variance_weighted' value which is deprecated since 0.17, it will be changed to 'uniform_average' starting from 0.19.
  DeprecationWarning)


0.0

In [124]:
articles_all = db.articles

In [128]:
found = []
descriptors_names = ["general_online_descriptors", "descriptors", "online_descriptors", "taxonomic_classifiers"]
for desc in descriptors_names:
    cursor = articles_all.find({desc: "Vaccination and Immunization"})
    print cursor.count()
    for d in cursor:
        found.append(d)

1201
969
305
0


In [129]:
len(found)

2475

In [133]:
found_sorted = sorted(found, key=lambda x:x['ISODate'])

In [136]:
for d in found_sorted:
    print d['ISODate']

1994-01-11 00:00:00
1994-02-13 00:00:00
1994-02-22 00:00:00
1994-03-02 00:00:00
1994-03-06 00:00:00
1994-03-08 00:00:00
1994-03-13 00:00:00
1994-03-22 00:00:00
1994-04-17 00:00:00
1994-04-24 00:00:00
1994-05-01 00:00:00
1994-05-15 00:00:00
1994-05-30 00:00:00
1994-05-31 00:00:00
1994-05-31 00:00:00
1994-06-13 00:00:00
1994-06-14 00:00:00
1994-06-18 00:00:00
1994-06-18 00:00:00
1994-06-26 00:00:00
1994-06-28 00:00:00
1994-07-01 00:00:00
1994-07-05 00:00:00
1994-07-17 00:00:00
1994-08-13 00:00:00
1994-08-23 00:00:00
1994-08-24 00:00:00
1994-09-03 00:00:00
1994-09-22 00:00:00
1994-09-25 00:00:00
1994-09-27 00:00:00
1994-10-10 00:00:00
1994-10-30 00:00:00
1994-11-01 00:00:00
1994-11-27 00:00:00
1994-11-29 00:00:00
1994-11-29 00:00:00
1995-01-19 00:00:00
1995-01-24 00:00:00
1995-02-19 00:00:00
1995-02-23 00:00:00
1995-03-17 00:00:00
1995-03-18 00:00:00
1995-03-25 00:00:00
1995-03-26 00:00:00
1995-04-01 00:00:00
1995-04-09 00:00:00
1995-04-13 00:00:00
1995-04-16 00:00:00
1995-04-30 00:00:00
